# Qgram
- 단어를 q 개씩 쪼갠다
- 각 쪼갠 단어를 카운트하여 백터화 한다 
- 백터 셋에서 공통된 칼럼의 수를 센다 (matrix.minimum(matrix))
- 전체 벡터 수로 나눈다
- 그게 바로 유사도! 

In [ ]:
def qgram_similarity(s1, s2, include_wb=True, ngram=(2, 2)):

#     if len(s1) != len(s2):
#         raise ValueError('Arrays or Series have to be same length.')

#     if len(s1) == len(s2) == 0:
#         return []

    # include word boundaries or not
    analyzer = 'char_wb' if include_wb is True else 'char'

    # prepare data
    data = s1.append(s2).fillna('')

    # The vectorizer
    vectorizer = CountVectorizer(
        analyzer=analyzer, 
        strip_accents='unicode', 
        ngram_range=ngram
    )

    vec_fit = vectorizer.fit_transform(data)

    def _metric_sparse_euclidean(u, v):

        match_ngrams = u.minimum(v).sum(axis=1)
        total_ngrams = np.maximum(u.sum(axis=1), v.sum(axis=1))

        # division by zero is not possible in our case, but 0/0 is possible.
        # Numpy raises a warning in that case.
        return np.true_divide(match_ngrams, total_ngrams).A1

    return _metric_sparse_euclidean(vec_fit[:len(s1)], vec_fit[len(s1):])


In [7]:
import pandas as pd

In [4]:
data = {'CO_NAME_new_x': {41: '중도석유 월드컵주유소',
  163: '더？푸드',
  512: '월드마트',
  659: '태화마트 삼척점',
  800: '굿모닝마트',
  869: '백조하이퍼마켓',
  975: '청명수산',
  1504: '명동하나이비인후과의원',
  1539: '계절밥상 대전갤러리아타임월드점',
  1808: '현대청과'},
 'CO_NAME_new_y': {41: '중도석유 월드컵주유소',
  163: '덕골생고기',
  512: '월드마트',
  659: '태화마트',
  800: '굿모닝마트',
  869: '백조한우마을',
  975: '청명수산',
  1504: '명동하나이비인후과의원',
  1539: '계절밥상대전갤러리아타임월드점',
  1808: '현대청과'},
 'exactssame': {41: True,
  163: False,
  512: True,
  659: False,
  800: True,
  869: False,
  975: True,
  1504: True,
  1539: False,
  1808: True},
 'subset': {41: True,
  163: False,
  512: True,
  659: True,
  800: True,
  869: False,
  975: True,
  1504: True,
  1539: False,
  1808: True}}

In [15]:
sample = pd.DataFrame(data)

In [16]:
sample

,CO_NAME_new_x,CO_NAME_new_y,exactssame,subset
41,중도석유 월드컵주유소,중도석유 월드컵주유소,True,True
163,더？푸드,덕골생고기,False,False
512,월드마트,월드마트,True,True
659,태화마트 삼척점,태화마트,False,True
800,굿모닝마트,굿모닝마트,True,True
869,백조하이퍼마켓,백조한우마을,False,False
975,청명수산,청명수산,True,True
1504,명동하나이비인후과의원,명동하나이비인후과의원,True,True
1539,계절밥상 대전갤러리아타임월드점,계절밥상대전갤러리아타임월드점,False,False
1808,현대청과,현대청과,True,True


In [17]:
sample_append = sample['CO_NAME_new_x'].append(sample['CO_NAME_new_y']).fillna('')

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
import numpy as np

In [11]:
vectorizer = CountVectorizer(
        analyzer='char_wb', 
        strip_accents=None, 
        ngram_range=(2, 2)
    )

In [18]:
vec_fit = vectorizer.fit_transform(sample_append)

In [20]:
u = vec_fit[:len(sample['CO_NAME_new_x'])]
v = vec_fit[len(sample['CO_NAME_new_x']):]

In [23]:
match_ngrams = u.minimum(v).sum(axis=1)
total_ngrams = np.maximum(u.sum(axis=1), v.sum(axis=1))

In [33]:
print(match_ngrams.A1)
print(total_ngrams.A1)
print(np.true_divide(match_ngrams, total_ngrams).A1)

[12  0  5  5  6  2  5 12 15  5]
[12  6  5  9  6  8  5 12 17  5]
[1.         0.         1.         0.55555556 1.         0.25
 1.         1.         0.88235294 1.        ]


In [90]:
qgrams_of_0_x = pd.DataFrame(vec_fit.toarray(), columns = vectorizer.get_feature_names()).iloc[3]
qgrams_of_0_y = pd.DataFrame(vec_fit.toarray(), columns = vectorizer.get_feature_names()).iloc[13]

In [91]:
qgrams_of_0_y[qgrams_of_0_y >=1]

 태    1
마트    1
태화    1
트     1
화마    1
Name: 13, dtype: int64

In [92]:
qgrams_of_0_x[qgrams_of_0_x >=1]

 삼    1
 태    1
마트    1
삼척    1
점     1
척점    1
태화    1
트     1
화마    1
Name: 3, dtype: int64

# longest_similarity

In [103]:
phone = {'REP_PHONE_NUM_new_x': {30: '025563774',
  202: '023346617',
  309: '0324466288',
  341: '0426729555',
  718: '0448661753',
  1020: '0318360435',
  1590: '0424722203',
  1594: '0315749274',
  1855: '0542567877',
  1909: '0424775475'},
 'REP_PHONE_NUM_new_y': {30: '0200000000',
  202: '028735005',
  309: '0324466288',
  341: '0426729555',
  718: '0448661753',
  1020: '0318360435',
  1590: '0424722203',
  1594: '0315749274',
  1855: '0542766676',
  1909: '0424775475'}}
phone = pd.DataFrame(phone)
conc = pd.Series(list(zip(phone['REP_PHONE_NUM_new_x'], phone['REP_PHONE_NUM_new_y'])))

In [105]:
conc

0     (025563774, 0200000000)
1      (023346617, 028735005)
2    (0324466288, 0324466288)
3    (0426729555, 0426729555)
4    (0448661753, 0448661753)
5    (0318360435, 0318360435)
6    (0424722203, 0424722203)
7    (0315749274, 0315749274)
8    (0542567877, 0542766676)
9    (0424775475, 0424775475)
dtype: object

In [93]:
def lcs_iteration(x):
        str1 = x[0]
        str2 = x[1]

        if str1 is np.nan or str2 is np.nan or min(len(str1), len(str2)) < min_len:
            longest = 0
            new_str1 = None
            new_str2 = None
        else:
            # Creating a matrix of 0s for preprocessing
            m = [[0] * (1 + len(str2)) for _ in range(1 + len(str1))]

            # Track length of longest substring seen
            longest = 0

            # Track the ending position of this substring in str1 (x) and str2(y)
            x_longest = 0
            y_longest = 0

            # Create matrix of substring lengths
            for x in range(1, 1 + len(str1)):
                for y in range(1, 1 + len(str2)):
                    # Check if the chars match
                    if str1[x - 1] == str2[y - 1]:
                        # add 1 to the diagonal
                        m[x][y] = m[x - 1][y - 1] + 1
                        # Update values if longer than previous longest substring
                        if m[x][y] > longest:
                            longest = m[x][y]
                            x_longest = x
                            y_longest = y
                    else:
                        # If there is no match, start from zero
                        m[x][y] = 0

            # Copy str1 and str2, but subtract the longest common substring
            # for the next iteration.
            new_str1 = str1[0:x_longest-longest]+str1[x_longest:]
            new_str2 = str2[0:y_longest-longest]+str2[y_longest:]

        return (new_str1, new_str2), longest


In [135]:
x = ('0424775475', '0423775475')

In [136]:
str1 = x[0]
str2 = x[1]
min_len = 2

In [137]:
# 둘 중 하나라도 비어있거나, 
if str1 is np.nan or str2 is np.nan or min(len(str1), len(str2)) < min_len:
    longest = 0
    new_str1 = None
    new_str2 = None

else:
    # Creating a matrix of 0s for preprocessing
    #x1의 길이+1 by x2의 길이+1의 0으로 된 매트릭스를 만든다. 
    m = [[0] * (1 + len(str2)) for _ in range(1 + len(str1))]

    # Track length of longest substring seen
    longest = 0

    # Track the ending position of this substring in str1 (x) and str2(y)
    x_longest = 0
    y_longest = 0
    
    # Create matrix of substring lengths
    for x in range(1, 1 + len(str1)):
        for y in range(1, 1 + len(str2)):
            # Check if the chars match
            if str1[x - 1] == str2[y - 1]:
                # add 1 to the diagonal
                m[x][y] = m[x - 1][y - 1] + 1
                # Update values if longer than previous longest substring
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
                    y_longest = y
            else:
                # If there is no match, start from zero
                m[x][y] = 0

    # Copy str1 and str2, but subtract the longest common substring
    # for the next iteration.
    new_str1 = str1[0:x_longest-longest]+str1[x_longest:]
    new_str2 = str2[0:y_longest-longest]+str2[y_longest:]

In [155]:
print(list(str1))
print(list(str2))

['0', '4', '2', '4', '7', '7', '5', '4', '7', '5']
['0', '4', '2', '3', '7', '7', '5', '4', '7', '5']


In [156]:
len(str2)

10

In [154]:
longest=0
for x in range(1, 1 + len(str1)):
    for y in range(1, 1 + len(str2)):
        # Check if the chars match
            if str1[x - 1] == str2[y - 1]:
                print(str1[x - 1], str2[y - 1], "are matched")
                # add 1 to the diagonal
                m[x][y] = m[x - 1][y - 1] + 1
                # Update values if longer than previous longest substring
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
                    y_longest = y
                    print("longest? ", longest, x, y)
#             else: print(str1[x - 1], str2[y - 1], "are diff")

0 0 are matched
longest?  1 1 1
4 4 are matched
longest?  2 2 2
4 4 are matched
2 2 are matched
longest?  3 3 3
4 4 are matched
4 4 are matched
7 7 are matched
7 7 are matched
7 7 are matched
7 7 are matched
7 7 are matched
7 7 are matched
5 5 are matched
5 5 are matched
4 4 are matched
4 4 are matched
longest?  4 8 8
7 7 are matched
7 7 are matched
7 7 are matched
longest?  5 9 9
5 5 are matched
5 5 are matched
longest?  6 10 10


In [115]:
m = [[0] * (1 + len(str2)) for _ in range(1 + len(str1))] #x1의 길이+1 by x2의 길이+1의 0으로 된 매트릭스를 만든다. 

In [121]:
list(range(1+len(str1)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [124]:
for _ in range(1+len(str1)):
    print([0] * (1 + len(str2)))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [126]:
1 + len(str1)

10

In [125]:
1 + len(str2)

11

m = [[0] * (1 + len(str2)) for _ in range(1 + len(str1))]